# 4. Command Line Question

The executable script file and the output screen can be found in the repository under the name CommandLine.sh and CommandLine.png. Below I put the text of the file for easier reading and understanding.

```bash
#!/bin/bash
input_file="vodclickstream_uk_movies_03.csv"

# 1. What is the most-watched Netflix title?
# group by the uniques titles and count the number of occurrences, saving them in a txt file
awk -F ',' '{print $4}' $input_file | sort | uniq -c | sort -nr > title_count.txt
# get the title from the txt file
most_watched_title=$(head -n 1 title_count.txt | awk '{$1=""; print $0}' | sed 's/^ *//')
echo "Most watched title: $most_watched_title"

# 2. The average time of subsequent clicks on Netflix.com
# filtering the duration tha has to be >= 0.0 and saving the values in a new file
awk -F ',' '{ if ($3 >= 0.0) print $0 }' $input_file > filtered_duration.csv
# the sum of duration (column 3) and the average of them
avg_duration=$(awk -F ',' '{print $3}' filtered_duration.csv | awk '{sum += $1} END {print sum/NR}')
# convert the duration from seconds to minutes
avg_min=$(awk -v avg_duration="$avg_duration" 'BEGIN { avg_min = avg_duration / 60; printf "%.2f\n", avg_min }')
# print the result
echo "Average duration of subsequent clicks: $avg_min minutes"

# 3. ID of the user that has spent the most time on Netflix
# group by the user_id and sum the duration time of each user, then sorting them and saving them in a txt file
awk -F ',' 'NR>1 { sum[$NF] += $3 } END { for (i in sum) print i, sum[i] }' filtered_duration.csv | sort -k2,2nr > sorted_users.txt
# get the user_id from the txt file
high_time_user=$(head -n 1 sorted_users.txt | awk '{print $1}')
echo "The user that has spent the most time on Netflix: $high_time_user"
```

1. To get the most watched Netflix title, we first grouped by each title in the dataset, then counted each time this appeared and saved the results in descending order in a new file, called ```title_count.txt```. At the end we extracted the first title of the file previusly created.
2. The average time of subsequent clicks on Netflix, we interpreted it as the average of all the duration times in the column 'duration' of the dataset, since it represents how much time the user stayed on the title, so each value corresponds to one click. Sais this, we filtered the dataset taking into account only the durations that are not negative, then summed all the remaining and took the average. Finally we converted the result from seconds to minutes.
3. The id of the user that has spent the most time on netflix has been extracted by first grouping the dataset by each user id, then summing the durations of each one and sorting them in descending order in a file, called ```sorted_users.txt```. At the end we extracted the first user in the file just created.

The following are the results

![CommandLine](CommandLine.png)

# 5. Algorithmic Question

He was given an initial personal score of when he enrolled ***S***, which changes every time he takes an exam.\
Every of the exams he has to take is assigned a mark ***p***.\
Once he has chosen an exam, his score becomes equal to the mark **S = p**, and at the same time, the scoring system changes:
- If he takes an "easy" exam (the score of the exam being less than his score), every other exam's mark is increased by
the quantity **$S-p$**.
- If he takes a "hard" exam (the score of the exam is greater than his score), every other exam's mark is decreased by
the quantity **$p-S$**.
- The exam score can't be negative, so if in any update a mark goes below zero it must be collapsed to zero.

He wants to know which is the highest score possible he could get.


In [1]:
import itertools

In [2]:
def FedericosScore(S, marks, p):

    copy_marks = marks.copy()                                             # T(n)

    if len(copy_marks) == 1:   # base case                                # T(1)
        return copy_marks[0]                                              

    if p <= S: # easy exam                                                # T(1)
        copy_marks.pop(0)                                                 # T(n)
        copy_marks = [x+(S-p) if x+(S-p)>=0 else 0 for x in copy_marks]   # T(n) 
        S = p                                                             # T(1)

    else: # hard exam                                                     # T(n)
        copy_marks.pop(0)
        copy_marks = [x-(p-S) if x-(p-S) >= 0 else 0 for x in copy_marks] #T(n) 
        S = p
        
    return FedericosScore(S, copy_marks, copy_marks[0])                   # T(n-1) 


In [4]:
# input 1
S = 8
marks = [5, 7, 1]

permutations = [list(x) for x in itertools.permutations(marks, len(marks))] # all the possible permutations of the exams O(n!)

scores = {}
for permu in permutations:
    scores[FedericosScore(S=S, marks=permu, p=permu[0])] = permu
max_s = max(scores.keys())
print(f'The highest score Federico can get is: {max_s} with this oder: {scores[max_s]}')

The highest score Federico can get is: 11 with this oder: [7, 1, 5]


In [5]:
# input 2
S = 25
marks = [18, 24, 21, 32, 27]

#FedericosScore(S, marks, marks[0])

permutations = [list(x) for x in itertools.permutations(marks, len(marks))] # all the possible permutations of the exams 

scores = {}
for permu in permutations:
    scores[FedericosScore(S=S, marks=permu, p=permu[0])] = permu
max_s = max(scores.keys())
print(f'The highest score Federico can get is: {max_s} with this order: {scores[max_s]}')

The highest score Federico can get is: 44 with this order: [27, 21, 32, 18, 24]


In [ ]:
# input 3
S = 30
marks = [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]

permutations = [list(x) for x in itertools.permutations(marks, len(marks))] # all the possible permutations of the exams 

# memory exceeded

scores = {}
for permu in permutations:
    scores[FedericosScore(S=S, marks=permu, p=permu[0])] = permu
max_s = max(scores.keys())
print(f'The highest score Federico can get is: {max_s} with this order: {scores[max_s]}')

The idea behind this initial solution is to have a function, ```FedrericosScore```, that takes in input:
- ```S```: the initial score of the student
- ```marks```: the list of the exams' marks
- ```p```: the first exam's mark

Then, since the student can choose the order of the esams he can take, we compute all the possible $n!$ permutations of the exams and give them in input, one by one, to the function.\
The result will be stored in a dictionary with keys the score and as vaulues the specific permutation that will make the student obstain that final score.\
Finally, we take the maximum score, out of the dictionary's keys, and its associated value; in this way the student will know in which order he has to take its exams in order to obtain the maximum score possible.

time complexity of the function:\
Assuming $n$ is the lenght of the ```marks``` list
- ```copy_marks``` has a time complexity of $T(n)$
- ```if len(copy_marks) == 1``` has a constant time complexity $T(1)$
- ```if p <= S:``` has a total time complexity of $T(n)$, since it has to pop the first element, shifting all the others, and in the list comprehension has to iterate on all the elements of the list.
- ```else:``` has a total time complexity of $T(n)$ for the same reason above.
- ```return FedericosScore``` has a time complexity of $T(n-1)$ in the worst case, since depends on the number of elements left in the list.

The total running time $T(n)$ can be expressed as:
\begin{align*}
T(n) =
\begin{cases} 
n+1 & \text{ if $n = 1$} \\
T(n-1)+5n+1 & \text{ if $n>1$} 
\end{cases}
\end{align*}
Generalizing we'll have a $T(n-i)+(5 \cdot i)n + i$, which at the end will result in $T(n) = 5n^2 - 3n + 2 \implies O(n^2)$.  

So the function has a quadratic time complexity of $O(n^2)$, then it has to be applied $n!$ times, for each one of the permutations of the list, so at the end the whole algorithm will have a complexity of $O(n^2 \cdot n!)$

This can't be sustainbable, since is highly memory-comsuming computing all the $n!$ permutations of a list as $n$ grows; for this reason this method doesn't work on the ```intput 3``` since has $n=12$ elements and all its permutations will be $479.001.600$. 

## Optimization

In [7]:
def Federicos_score_opt(S, marks):

    if len(marks) == 0:                                                                # T(1)     
        return S                                                                       # T(1)     
    
    maxScore = 0  # keep track of the maximum score Federico can achieve               # T(1)
    for i, mark in enumerate(marks):                                                   # T(n) 
        update_marks = marks[:i] + marks[i+1:] # don't consider the current exam       # T(n) 
        newScore = mark                                                                # T(1)
        if mark <= S: # esasy exam                                                     # T(1)
            update_marks = [x+(S-mark) if x+(S-mark)>=0 else 0 for x in update_marks]  # T(n)
        else: # hard exam
            update_marks = [x-(mark-S) if x-(mark-S)>=0 else 0 for x in update_marks]  # T(n)

        # comparing the current maxScore with the result of the recursive call.
        maxScore = max(maxScore, Federicos_score_opt(newScore, update_marks))          # T(n-1)
    return maxScore                                                                    # T(1)

In [8]:
# input 1
S_1 = 8
p_1 = [5, 7, 1]
result_1 = Federicos_score_opt(S_1, p_1)
print("Output 1:", result_1)

Output 1: 11


In [12]:
S_2 = 25
p_2 = [18, 24, 21, 32, 27]
result_2 = Federicos_score_opt(S_2, p_2)
print("Output 2:", result_2)

Output 2: 44


In [11]:
S_3 = 30
p_3 = [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]
result_3 = Federicos_score_opt(S_3, p_3)
print("Output 3:", result_3)

# took 16 mins

Output 3: 109


In this solution we don't have to explicitally compute all the possible permutations of the exams, because the optimized function explores all possible combinations of the exams recursivly and eventually finds the combination that guarantees the highest score possible by exploring different paths for each exam in the list. 

Time complexity of the function:\
Assuming $n$ is the lenght of the ```marks``` list
- ```if len(copy_marks) == 1``` has a constant time complexity $T(1)$
- ```for i, mark in enumerate(marks)``` has a linear time of $T(n)$ because we iterate through all the exams
- ```update_marks = marks[:i] + marks[i + 1:]``` has a linear time complexity $T(n)$ since we're performing a list slicing
- ```if mark <= S``` loop has a total complexity of $T(n)$ since the list comprehension inside explores all the elements in the list
- ```else``` loop has the complexity of $T(n)$ for the same reason above
- ```maxScore = max(maxScore, Federicos_score_opt(newScore, update_marks))``` has a time complexity of $T(n-1)$ in the worst case, since depends on the number of elements left in the list.

Since the recursive call of the function is inside the loop of complexity $O(n^2)$, this significantly contributes to the overall complexity of the function, that is $O(n^3)$.\
This is an improvement in respect to the previus algorithm, both on time complexity and memory complexity, since we're now able to actually see a result for the exam's list of the third input.

# ChatGPT optimization implementation 

In [31]:
def Federicos_score_chatGPT(S, marks, store={}):
    if len(marks) == 0:
        return S

    if (S, tuple(marks)) in store:
        return store[(S, tuple(marks))]

    maxScore = 0
    for i, mark in enumerate(marks):
        update_marks = marks[:i] + marks[i + 1:]
        newScore = mark
        if mark <= S:
            update_marks = [x + (S - mark) if x + (S - mark) >= 0 else 0 for x in update_marks]
        else:
            update_marks = [x - (mark - S) if x - (mark - S) >= 0 else 0 for x in update_marks]

        maxScore = max(maxScore, Federicos_score_chatGPT(newScore, update_marks, store))

    store[(S, tuple(marks))] = maxScore
    return maxScore


In [32]:
# input 1
S_1 = 8
p_1 = [5, 7, 1]
result_1 = Federicos_score_chatGPT(8, [5,7,1])
print("Output 1:", result_1)

Output 1: 11


In [29]:
S_2 = 25
p_2 = [18, 24, 21, 32, 27]
result_2 = Federicos_score_chatGPT(S_2, p_2)
print("Output 2:", result_2)

Output 2: 44


In [30]:
S_3 = 30
p_3 = [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]
result_3 = Federicos_score_chatGPT(S_3, p_3)
print("Output 3:", result_3)

# took 2.5 seconds

Output 3: 109


Optimization Approach:

- **Memoization**: Introduced a memoization dictionary (```store```) to store previously computed results for combinations of ```S``` and ```marks```. This prevents redundant calculations by checking if a specific combination has been encountered before. If so, it retrieves the result from memo instead of recomputing it.

- **Avoidance of Recomputation**: By storing and retrieving results from the ```store``` dictionary, the function bypasses the need to recompute solutions for the same combination of ```S``` and ```marks```. This significantly reduces the number of recursive calls and computations required, leading to a more efficient algorithm overall.

This optimization effectively reduces the time complexity of the algorithm from $O(n^3)$ to $O(n^2)$ by avoiding redundant computations through memoization.